# Proyecto Final: Sistema de Salud Colombiano

### Motivación 

Aportar al entendimiento de las desigualdades en el sistema de salud colombiano, respondiendo a las siguientes líneas de trabajo:

- Comparación entre cobertura (régimen subsidiado/contributivo) y distribución de las IPS a nivel municipal

- Acceso a nivel municipal a IPS de acuerdo con el nivel de complejidad

- Asistencia de las personas a las IPS cuando tienen una enfermedad que no requiere hospitalización 

- Percepción de las personas sobre la calidad del servicio de la IPS


In [1]:
#Importar la librería necesarias para ejecutar el notebook
!pip install sodapy
from sodapy import Socrata
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import re

import os, sys

#conozca el directorio de trabajo actual
os.getcwd()

#Para utilizar SODA debe utlizar su correo y contraseña de Datos abiertos.


'C:\\Users\\ktaza\\OneDrive\\Documentos\\MEPP'

### Descargar la base da datos de Datos Abiertos Colombia

Para acceder a la base de datos de las IPS en Colombia, se va a utilizar el protocolo Socrata Open Data API (SODA) de Datos abiertos. Esta base contiene las siguientes variables relevantes para este trabajo:
- Departamente (código y nombre)
- Municipio (código y nombre)
- najunombre si la IPS es privada o pública
- código de habilitación es el id de cada IPS-Sede
- NIT identifica la IPS

In [2]:
#Acceso a la información requerida
MyAppToken = '40rPvfBZ93PP5EuqIdzUeF0GJ'
client = Socrata('www.datos.gov.co',
                  MyAppToken,
                  username="correo usuario",
                  password="contraseña")

data = client.get("c36g-9fc2", content_type="csv", limit=10926) 

#Transforar la base de datos a dataframe de pandas
data_ips = pd.DataFrame.from_records(data)
data_ips.columns = data_ips.iloc[0]
data_ips.drop(data_ips.index[0], inplace = True)
data_ips

,coddepartamento,departamento,codmunicipio,muninombre,codigohabilitacion,nombre,nitsnit,ese,direccion,telefono,email,najunombre,numerosedeprincipal,fecha_corte_reps
1,91,Amazonas,91001,LETICIA,9100100019,E.S.E. HOSPITAL SAN RAFAEL DE LETICIA,838000096,SI,Carrera 10 NO. 13-78 Leticia (Amazonas),3203048903-3204650596,gerencia@esehospitalsanrafael-leticia-amazonas...,Pública,1,Fecha corte REPS: Mar 6 2021 9:12PM
2,91,Amazonas,91001,LETICIA,9100100185,FREYA SKIN AND BODY CARE S.A.S TWO,901438650,,CRA.8 A No.8-56,3186435065,freya.skinandbodycare@gmail.com,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
3,91,Amazonas,91001,LETICIA,9100100069,FUNDACION CLINICA LETICIA,900142282,NO,AK INTERNACIONAL No 6-05,985927718,direccion@fundacionclinicaleticia.com.co,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
4,91,Amazonas,91001,LETICIA,9100100158,IMPROVE QUALITY REDUCE COST SAVE LIFES AUDITOR...,900784482,,CRA. 6 No. 8 - 76,3104769842,servisaludqcl@gmail.com,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
5,91,Amazonas,91001,LETICIA,9100100144,"INSTITUTO PARA LA PREVENCIÓN, EL TRATAMIENTO Y...",900275974,,CARRERA 4 No. 1 - 55 Barrio Colombia,5925096,renaceraipsas@gmail.com,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10921,99,Vichada,99001,PUERTO CARREÑO,9900100099,CRUZ ROJA COLOMBIANA SECCIONAL VICHADA,842000133,,CARRERA 11 No 19-93 BARRIO LAS ACACIAS,3114476340 3105810358,doctrinavichada@cruzrojacolombiana.org,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
10922,99,Vichada,99001,PUERTO CARREÑO,9900100006,E.S.E. HOSPITAL DEPARTAMENTAL SAN JUAN DE DIOS,842000004,SI,CL 18 No 10-43,(098) 5654009/5654438,gerencia@esesanjuandediospc.gov.co,Pública,1,Fecha corte REPS: Mar 6 2021 9:12PM
10923,99,Vichada,99001,PUERTO CARREÑO,9900100139,GRUPO DE GESTION SOCIAL GGS SAS,901226069,,CARRERA 10 No. 13-50 (LC 01),3505956221,ggs.vichada@hotmail.com,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM
10924,99,Vichada,99001,PUERTO CARREÑO,9900100117,IPS DIPOSALUD SAS,900653426,,CARRERA 12 # 26-30,3138543185,dipodent@yahoo.com,Privada,1,Fecha corte REPS: Mar 6 2021 9:12PM


Para acceder a el nivel de complejidad que tiene la ips se importará otra base de datos de datos abiertos que contiene la información sobre el listado de IPS en Colombia según su nivel de complejidad. Es importante hacer el merge entre las dos bases de datos para consolidar una base da datos que tenga las variables mencionadas inicialmente y el nivel de atención. A pesar de que esta base de datos tiene una infomación similar a la anterior, esta no cuenta con el código departamental y municipal indispensable para realizar cruces de información con el censo del DANE.

Las llaves para unir las dos bases de datos son: código de habilitación y nit

In [3]:
#Acceso a la información del nivel de la IPS
MyAppToken = '40rPvfBZ93PP5EuqIdzUeF0GJ'
client = Socrata('www.datos.gov.co',
                  MyAppToken,
                  username="correo usuario",
                  password="contraseña")

data = client.get("ugc5-acjp", content_type="csv", limit=10926) 

#Transforar la base de datos a dataframe de pandas
data_nivel = pd.DataFrame.from_records(data)
data_nivel.columns = data_nivel.iloc[0]
data_nivel = data_nivel.drop(data_nivel.index[0])


#Eliminar las variables que  no se usarán para el merge
data_nivel = data_nivel[["codigo_habilitacion","nits_nit","nivel"]]
data_nivel

,codigo_habilitacion,nits_nit,nivel
1,9100100019,838000096,2
2,9100100185,901438650,
3,9100100069,900142282,
4,9100100158,900784482,
5,9100100144,900275974,
...,...,...,...
10921,9900100099,842000133,
10922,9900100006,842000004,2
10923,9900100139,901226069,
10924,9900100117,900653426,


In [4]:
#Merge entre data_ips y data_nivel
data_salud = pd.merge(data_ips, data_nivel, how='left', left_on = ["codigohabilitacion","nitsnit"], \
                      right_on = ["codigo_habilitacion","nits_nit"] )
data_salud["cantidad"] = 1

#Limpieza 
data_salud["nivel"].replace(to_replace = "", value = np.nan, inplace=True)

#Calculos a nivel municipal 

#Cantidad de IPS
data_cantidad = data_salud.groupby(["coddepartamento", "codmunicipio", "cantidad"]).size().unstack(fill_value=0)
data_cantidad = data_cantidad.reset_index().rename_axis(None).rename_axis(None, axis=1)
data_cantidad.rename(columns = { 1:"cantidad_ips"}, inplace= True)

#Cantidad de IPS públicas por nivel de complejidad
data_nivel2 = data_salud.groupby(["coddepartamento", "codmunicipio", "nivel"]).size().unstack(fill_value=0)
data_nivel2 = data_nivel2.reset_index().rename_axis(None).rename_axis(None, axis=1)
data_nivel2.rename(columns = { "1" : "nivel_1", "2" : "nivel_2", "3" : "nivel_3"}, inplace= True)

#Cantidad de IPS privadas, mixtas y públicas
data_tipo = data_salud.groupby(["coddepartamento", "codmunicipio", "najunombre"]).size().unstack(fill_value=0)
data_tipo = data_tipo.reset_index().rename_axis(None).rename_axis(None, axis=1)
data_tipo.rename(columns = { "Pública" : "ips_publica", "Privada" : "ips_privada" , "Mixta" : "ips_mixta"}, inplace= True)

#Merge de la información del municipio.
df1 = pd.merge(data_cantidad, data_nivel2, how='left', on= ["coddepartamento", "codmunicipio"])
data_salud = pd.merge(df1, data_tipo, how='left', on= ["coddepartamento", "codmunicipio"])
data_salud

,coddepartamento,codmunicipio,cantidad_ips,nivel_1,nivel_2,nivel_3,ips_mixta,ips_privada,ips_publica
0,05,05001,597,2.0,2.0,2.0,1,590,6
1,05,05002,1,1.0,0.0,0.0,0,0,1
2,05,05021,1,1.0,0.0,0.0,0,0,1
3,05,05030,1,1.0,0.0,0.0,0,0,1
4,05,05031,1,1.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...
921,95,95001,15,1.0,1.0,0.0,0,13,2
922,97,97001,3,1.0,1.0,0.0,0,1,2
923,99,99001,5,0.0,1.0,0.0,0,4,1
924,99,99624,1,NaN,NaN,NaN,0,1,0


### Procesar el componente de personas del Censo a nivel deparamental

Esta base de datos será utilizada para agregar al análisis características a nivel municipal, para este estudio nos interesan las siguientes variables sobre la población del censo: sexo, edad y si ha presentando problemas de salud. Esta última pregunta nos puede mostrar si las personas acuden a su IPS cuando tienen problemas de salud o lo hacen por consulta particular o métodos no ortodoxos.

In [5]:
#Indicar el directorio de trabajo donde guardó las bases de datos a utilizar:
os.chdir("C:\\Users\\ktaza\\OneDrive\\Documentos\\MEPP\\Métodos cumputacionales\\Censo")

# Crear una lista con los departamentos y Bogotá
deptos = [91, 81, 11, 13, 15, 17, 18, 85, 19, 20, 27, 23, 25, 94, 95, 41, 44, 47, 50, 52, 54, 86, 63, 66, 88, 68, \
          70, 73, 76, 97, 99]
deptos2 = [5, 8]

In [6]:
#Dado que por capacidad no se puede trabajar con la totalidad del censo, se realizó un loop con cada una de las bases de datos
#a nivel departamental, para cada caso se importaron las columnas de interés, posteriormente se realizó la limpieza y cáculo a 
#nivel municipal para cada variable. Este loop es para deptos de dos dígitos superiores a 0.

for i in deptos:
    #Importar base de datos
    col_list = ["U_DPTO", "U_MPIO", "P_SEXO", "P_EDADR", "P_ENFERMO", "P_QUEHIZO_PPAL", "PA_LO_ATENDIERON", "PA1_CALIDAD_SERV"]
    data = pd.read_csv("CNPV2018_5PER_A2_" + str(i) + ".csv", usecols = col_list, converters = {"U_DPTO" : str, "U_MPIO": str})

    #Limpieza base de datos

    #Sexo
    #1 - Hombre, 2 - Mujer. Convertir mujer igual 0 

    #P_EDAD
    #1 de 00 A 04 Años
    #2 de 05 A 09 Años
    #3 de 10 A 14 Años
    #4 de 15 A 19 Años
    #5 de 20 A 24 Años
    #6 de 25 A 29 Años
    #7 de 30 A 34 Años
    #8 de 35 A 39 Años
    #9 de 40 A 44 Años
    #10 de 45 A 49 Años
    #11 de 50 A 54 Años
    #12 de 55 A 59 Años
    #13 de 60 A 64 Años
    #14 de 65 A 69 Años
    #15 de 70 A 74 Años
    #16 de 75 A 79 Años
    #17 de 80 A 84 Años
    #18 de 85 A 89 Años
    #19 de 90 A 94 Años
    #20 de 95 A 99 Años
    #21 de 100 y más Años
    data["P_EDADR"].unique()

    #P_ENFERMO ¿Algun problema de salud en los últimos 30 días, sin hospitalización?
    # 1 Sí, 2 No, 9 No informa
    data["P_ENFERMO"].replace(to_replace = 9, value = np.nan, inplace=True)
    data["P_ENFERMO"].unique()

    #P_QUEHIZO_PPAL Tratamiento principal del problema de salud
    #1 Acudió a la entidad de seguridad social en salud de la cual es filiado(a)?
    #2 Acudió a un médico particular? (general, especialista, odontólogo, terapeuta u otro)
    #3 Acudió a un boticario, farmacéuta, droguista?
    #4 Asistió a terapias alternativas? (acupuntura, esencias florales, musicoterapias, homeópata, etc.)
    #5 Acudió a una autoridad indígena espiritual?
    #6 Acudió a otro médico de un grupo étnico? (curandero, yerbatero, etc.)
    #7 Usó remedios caseros?
    #8 Se autorrecetó
    #9 No hizo nada
    #99 No Informa
    #No Aplica

    data["P_QUEHIZO_PPAL"].replace(to_replace = 99, value = np.nan, inplace=True)
    data["P_QUEHIZO_PPAL"].replace(to_replace = 0, value = np.nan, inplace=True)

    #PA_LO_ATENDIERON Atención del problema de salud en la IPS
    #1 1 Sí, 2 No, 9 No Informa, No Aplica

    data["PA_LO_ATENDIERON"].replace(to_replace = 9, value = np.nan, inplace=True)

    #PA1_CALIDAD_SERV Calidad de la prestación del servicio de salud en la IPS
    #1 Muy bueno
    #2 Bueno
    #3 Malo
    #4 Muy Malo
    #9 No Informa
    #No Aplica
    data["PA1_CALIDAD_SERV"].replace(to_replace = 9, value = np.nan, inplace=True)
    data["PA1_CALIDAD_SERV"].replace(to_replace = 0, value = np.nan, inplace=True)

    #Bases de datos con las caracterísiticas para hacer merge

    #Sexo
    data_sexo = data.groupby(["U_DPTO", "U_MPIO", "P_SEXO"]).size().unstack(fill_value=0)
    data_sexo = data_sexo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_sexo.rename(columns = {1:"Hombre", 2: "Mujer"}, inplace= True)

    #P_EDAD
    data_edad = data.groupby(["U_DPTO", "U_MPIO", "P_EDADR"]).size().unstack(fill_value=0)
    data_edad = data_edad.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_edad.rename(columns = {1 : "00_04_anios" , 2 : "05_09_anios" , 3 : "10_14_anios" , 4 : "15_19_anios" , \
                                5 : "20_24_anios" , 6 : "25_29_anios" , 7 : "30_34_anios" , 8 : "35_39_anios" , \
                                9 : "40_44_anios" , 10 : "45_49_anios" , 11 : "50_54_anios" , 12 : "55_59_anios" , \
                                13 : "60_64_anios" , 14 : "65_69_anios" , 15 : "70_74_anios" , 16 : "75_79_anios" , \
                                17 : "80_84_anios" , 18 : "85_89_anios" , 19 : "90_94_anios" , 20 : "95_99_anios" , \
                                21 : "100_anios"}, inplace= True)
    #P_ENFERMO 
    data_enfermo = data.groupby(["U_DPTO", "U_MPIO", "P_ENFERMO"]).size().unstack(fill_value=0)
    data_enfermo = data_enfermo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_enfermo.rename(columns = { 1.0:"enfermo", 2.0: "no_enfermo"}, inplace= True)

    #P_QUEHIZO_PPAL 
    data_mecanismo = data.groupby(["U_DPTO", "U_MPIO", "P_QUEHIZO_PPAL"]).size().unstack(fill_value=0)
    data_mecanismo = data_mecanismo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_mecanismo.rename(columns = { 1 : "m_ips", 2 : "m_particular", 3 : "m_farmaceuta", \
                                     4 : "m_alternativa", 5 : "m_indigena", 6 : "m_curandero", \
                                     7 : "m_caseros", 8 : "m_autorreceta", 9 : "m_no_hizo"}, inplace= True)


    #PA_LO_ATENDIERON 
    data_atencion = data.groupby(["U_DPTO", "U_MPIO", "PA_LO_ATENDIERON"]).size().unstack(fill_value=0)
    data_atencion = data_atencion.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_atencion.rename(columns = { 1.0:"atencion_ips", 2.0: "no_atencion_ips"}, inplace= True)

    #PA1_CALIDAD_SERV Calidad de la prestación del servicio de salud en la IPS
    data_calidad = data.groupby(["U_DPTO", "U_MPIO", "PA1_CALIDAD_SERV"]).size().unstack(fill_value=0)
    data_calidad = data_calidad.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_calidad.rename(columns = { 1.0:"mbuen_servicio", 2.0: "buen_servicio", 3.0: "mal_servicio", 4.0: "mmal_servicio"},\
                        inplace= True)

    #Merge de la información del municipio.
    df1 = pd.merge(data_sexo, data_edad, how='inner', on= ["U_DPTO", "U_MPIO"])
    df2 = pd.merge(df1, data_enfermo, how='inner', on= ["U_DPTO", "U_MPIO"])
    df3 = pd.merge(df2, data_mecanismo, how='inner', on= ["U_DPTO", "U_MPIO"])
    df4 = pd.merge(df3, data_atencion, how='inner', on= ["U_DPTO", "U_MPIO"])
    data_dpto = pd.merge(df4, data_calidad, how='inner', on= ["U_DPTO", "U_MPIO"])
    #Guardar base de datos
    data_dpto.to_csv("data_dpto_" + str(i) + ".csv", index = False)
    del data_sexo, data_edad, data_enfermo, data_mecanismo, data_atencion, data_calidad, df1, df2, df3, df4


In [7]:
#Dado que por capacidad no se puede trabajar con la totalidad del censo, se realizó un loop con cada una de las bases de datos
#a nivel departamental, para cada caso se importaron las columnas de interés, posteriormente se realizó la limpieza y cáculo a 
#nivel municipal para cada variable. Este loop es para deptos de un dígito, pues empiezan con 0.

for i in deptos2:
    #Importar base de datos
    col_list = ["U_DPTO", "U_MPIO", "P_SEXO", "P_EDADR", "P_ENFERMO", "P_QUEHIZO_PPAL", "PA_LO_ATENDIERON", "PA1_CALIDAD_SERV"]
    data = pd.read_csv("CNPV2018_5PER_A2_0" + str(i) + ".csv", usecols = col_list, converters = {"U_DPTO" : str, "U_MPIO": str})
    
    #Limpieza base de datos

    #Sexo
    #1 - Hombre, 2 - Mujer. Convertir mujer igual 0 

    #P_EDAD
    #1 de 00 A 04 Años
    #2 de 05 A 09 Años
    #3 de 10 A 14 Años
    #4 de 15 A 19 Años
    #5 de 20 A 24 Años
    #6 de 25 A 29 Años
    #7 de 30 A 34 Años
    #8 de 35 A 39 Años
    #9 de 40 A 44 Años
    #10 de 45 A 49 Años
    #11 de 50 A 54 Años
    #12 de 55 A 59 Años
    #13 de 60 A 64 Años
    #14 de 65 A 69 Años
    #15 de 70 A 74 Años
    #16 de 75 A 79 Años
    #17 de 80 A 84 Años
    #18 de 85 A 89 Años
    #19 de 90 A 94 Años
    #20 de 95 A 99 Años
    #21 de 100 y más Años
    data["P_EDADR"].unique()

    #P_ENFERMO ¿Algun problema de salud en los últimos 30 días, sin hospitalización?
    # 1 Sí, 2 No, 9 No informa
    data["P_ENFERMO"].replace(to_replace = 9, value = np.nan, inplace=True)
    data["P_ENFERMO"].unique()

    #P_QUEHIZO_PPAL Tratamiento principal del problema de salud
    #1 Acudió a la entidad de seguridad social en salud de la cual es filiado(a)?
    #2 Acudió a un médico particular? (general, especialista, odontólogo, terapeuta u otro)
    #3 Acudió a un boticario, farmacéuta, droguista?
    #4 Asistió a terapias alternativas? (acupuntura, esencias florales, musicoterapias, homeópata, etc.)
    #5 Acudió a una autoridad indígena espiritual?
    #6 Acudió a otro médico de un grupo étnico? (curandero, yerbatero, etc.)
    #7 Usó remedios caseros?
    #8 Se autorrecetó
    #9 No hizo nada
    #99 No Informa
    #No Aplica

    data["P_QUEHIZO_PPAL"].replace(to_replace = 99, value = np.nan, inplace=True)
    data["P_QUEHIZO_PPAL"].replace(to_replace = 0, value = np.nan, inplace=True)

    #PA_LO_ATENDIERON Atención del problema de salud en la IPS
    #1 1 Sí, 2 No, 9 No Informa, No Aplica

    data["PA_LO_ATENDIERON"].replace(to_replace = 9, value = np.nan, inplace=True)

    #PA1_CALIDAD_SERV Calidad de la prestación del servicio de salud en la IPS
    #1 Muy bueno
    #2 Bueno
    #3 Malo
    #4 Muy Malo
    #9 No Informa
    #No Aplica
    data["PA1_CALIDAD_SERV"].replace(to_replace = 9, value = np.nan, inplace=True)
    data["PA1_CALIDAD_SERV"].replace(to_replace = 0, value = np.nan, inplace=True)

    #Bases de datos con las caracterísiticas para hacer merge

    #Sexo
    data_sexo = data.groupby(["U_DPTO", "U_MPIO", "P_SEXO"]).size().unstack(fill_value=0)
    data_sexo = data_sexo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_sexo.rename(columns = {1:"Hombre", 2: "Mujer"}, inplace= True)

    #P_EDAD
    data_edad = data.groupby(["U_DPTO", "U_MPIO", "P_EDADR"]).size().unstack(fill_value=0)
    data_edad = data_edad.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_edad.rename(columns = {1 : "00_04_anios" , 2 : "05_09_anios" , 3 : "10_14_anios" , 4 : "15_19_anios" , \
                                5 : "20_24_anios" , 6 : "25_29_anios" , 7 : "30_34_anios" , 8 : "35_39_anios" , \
                                9 : "40_44_anios" , 10 : "45_49_anios" , 11 : "50_54_anios" , 12 : "55_59_anios" , \
                                13 : "60_64_anios" , 14 : "65_69_anios" , 15 : "70_74_anios" , 16 : "75_79_anios" , \
                                17 : "80_84_anios" , 18 : "85_89_anios" , 19 : "90_94_anios" , 20 : "95_99_anios" , \
                                21 : "100_anios"}, inplace= True)
    #P_ENFERMO 
    data_enfermo = data.groupby(["U_DPTO", "U_MPIO", "P_ENFERMO"]).size().unstack(fill_value=0)
    data_enfermo = data_enfermo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_enfermo.rename(columns = { 1.0:"enfermo", 2.0: "no_enfermo"}, inplace= True)

    #P_QUEHIZO_PPAL 
    data_mecanismo = data.groupby(["U_DPTO", "U_MPIO", "P_QUEHIZO_PPAL"]).size().unstack(fill_value=0)
    data_mecanismo = data_mecanismo.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_mecanismo.rename(columns = { 1 : "m_ips", 2 : "m_particular", 3 : "m_farmaceuta", \
                                     4 : "m_alternativa", 5 : "m_indigena", 6 : "m_curandero", \
                                     7 : "m_caseros", 8 : "m_autorreceta", 9 : "m_no_hizo"}, inplace= True)


    #PA_LO_ATENDIERON 
    data_atencion = data.groupby(["U_DPTO", "U_MPIO", "PA_LO_ATENDIERON"]).size().unstack(fill_value=0)
    data_atencion = data_atencion.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_atencion.rename(columns = { 1.0:"atencion_ips", 2.0: "no_atencion_ips"}, inplace= True)

    #PA1_CALIDAD_SERV Calidad de la prestación del servicio de salud en la IPS
    data_calidad = data.groupby(["U_DPTO", "U_MPIO", "PA1_CALIDAD_SERV"]).size().unstack(fill_value=0)
    data_calidad = data_calidad.reset_index().rename_axis(None).rename_axis(None, axis=1)
    data_calidad.rename(columns = { 1.0:"mbuen_servicio", 2.0: "buen_servicio", 3.0: "mal_servicio", 4.0: "mmal_servicio"},\
                        inplace= True)

    #Merge de la información del municipio.
    df1 = pd.merge(data_sexo, data_edad, how='inner', on= ["U_DPTO", "U_MPIO"])
    df2 = pd.merge(df1, data_enfermo, how='inner', on= ["U_DPTO", "U_MPIO"])
    df3 = pd.merge(df2, data_mecanismo, how='inner', on= ["U_DPTO", "U_MPIO"])
    df4 = pd.merge(df3, data_atencion, how='inner', on= ["U_DPTO", "U_MPIO"])
    data_dpto = pd.merge(df4, data_calidad, how='inner', on= ["U_DPTO", "U_MPIO"])
    #Guardar base de datos
    data_dpto.to_csv("data_dpto_" + str(i) + ".csv", index = False)
    del data_sexo, data_edad, data_enfermo, data_mecanismo, data_atencion, data_calidad, df1, df2, df3, df4


In [8]:
#Unir las bases de datos guardadas en los dos loops realizados anteriormente

#lista de departamentos sin el depto #8
deptos = [91, 81, 11, 13, 15, 17, 18, 85, 19, 20, 27, 23, 25, 94, 95, 41, 44, 47, 50, 52, 54, 86, 63, 66, 88, 68, \
          70, 73, 76, 97, 99, 5]

#Abrir la base del depto #8:
data_municipio = pd.read_csv("data_dpto_8.csv", converters = {"U_DPTO" : str, "U_MPIO": str})

#Loop para realizar append de las bases de datos
for i in deptos:
    aux = pd.read_csv("data_dpto_" + str(i) + ".csv", converters = {"U_DPTO" : str, "U_MPIO": str})
    data_municipio = data_municipio.append(aux)
    
#Creación de el código del municipio completo para poder hacer el merge con las bases de datos de salud  
data_municipio["cod_mpio"] = data_municipio["U_DPTO"] + data_municipio["U_MPIO"] 


### Merge de las bases de datos de Datos Abiertos y DANE

Se realizará el merge entre la base de salud realizada inicialmente y el censo

In [11]:
#Merge
df_final = pd.merge(data_salud, data_municipio, how='right', left_on = ["coddepartamento","codmunicipio"], \
                    right_on = ["U_DPTO","cod_mpio"] )

#Unión con divipola para cambiar labels de los municipios
divipola = pd.read_csv("Divipola.csv", sep = ";", dtype = str)

df_final = pd.merge(df_final, divipola, how='left', left_on = ["U_DPTO","cod_mpio"], right_on = ["DEPTO","DIVIPOLA"] )
df_final.drop(["coddepartamento", "codmunicipio", "U_DPTO", "U_MPIO", "DIVIPOLA", "MUNICIPIO.1"], axis = 1, inplace = True)
df_final.rename(columns = { "DEPARTAMENTO":"dpto", "DEPTO": "code_dpto", "MUNICIPIO": "mpio"}, inplace= True)

#En caso de que su jupyter le impida ver todas las columnas corra la siguiente línea:
pd.set_option('max_columns', None)

#Limpieza de nulos en IPS (si es nulo es porque el sistema no tiene información por lo cual se podría pensar que no tiene
#IPS)
df_final[['cantidad_ips', 'nivel_1','nivel_2', 'nivel_3', 'ips_mixta', 'ips_privada', 'ips_publica']] = \
df_final[['cantidad_ips', 'nivel_1','nivel_2', 'nivel_3', 'ips_mixta', 'ips_privada', 'ips_publica']].fillna(value=0)

#Exportar base de datos
df_final.to_excel("data_final.xlsx",index = False)

#Visualizar base de datos
df_final


,cantidad_ips,nivel_1,nivel_2,nivel_3,ips_mixta,ips_privada,ips_publica,Hombre,Mujer,00_04_anios,05_09_anios,10_14_anios,15_19_anios,20_24_anios,25_29_anios,30_34_anios,35_39_anios,40_44_anios,45_49_anios,50_54_anios,55_59_anios,60_64_anios,65_69_anios,70_74_anios,75_79_anios,80_84_anios,85_89_anios,90_94_anios,95_99_anios,100_anios,enfermo,no_enfermo,m_ips,m_particular,m_farmaceuta,m_alternativa,m_indigena,m_curandero,m_caseros,m_autorreceta,m_no_hizo,atencion_ips,no_atencion_ips,mbuen_servicio,buen_servicio,mal_servicio,mmal_servicio,cod_mpio,dpto,mpio,code_dpto
0,572.0,0.0,1.0,1.0,1.0,569.0,2.0,536790,583313,79260,82171,84712,94976,98201,93347,85453,83642,69617,65152,66020,60317,48836,36984,25817,19183,13484,7759,3508,1149,515,62743,1040638,44979,5232,1817,102,3.0,5.0,2934,6358,1286,44081,898,6951,30663,5145,1322,08001,Atlantico,Barranquilla,08
1,13.0,1.0,0.0,0.0,0.0,12.0,1.0,30695,30832,4702,4981,5295,5508,5067,4926,4555,4393,3624,3582,3624,3280,2363,1874,1338,1032,772,357,183,56,15,6269,54747,4846,265,246,5,1.0,2.0,265,471,168,4802,43,423,3842,484,53,08078,Atlantico,Baranoa,08
2,9.0,1.0,0.0,0.0,0.0,8.0,1.0,11394,11416,2049,2183,2219,2130,1872,1765,1619,1357,1254,1225,1294,1057,794,574,392,441,319,185,56,18,7,667,22045,632,11,4,0,0.0,0.0,7,2,11,608,24,348,225,32,3,08137,Atlantico,Campo De La Cruz,08
3,2.0,1.0,0.0,0.0,0.0,1.0,1.0,7961,7670,1382,1612,1475,1500,1312,1223,1125,1001,870,920,837,686,505,389,309,222,151,73,34,4,1,939,14574,779,42,1,0,0.0,0.0,19,75,23,771,8,151,563,56,1,08141,Atlantico,Candelaria,08
4,7.0,1.0,0.0,0.0,0.0,6.0,1.0,27613,27510,4864,4883,4953,5234,4855,4605,4389,4125,3486,3209,2837,2390,1782,1288,877,586,392,214,93,36,25,1484,53274,1148,49,11,0,0.0,0.0,119,117,40,1113,35,238,744,97,34,08296,Atlantico,Galapa,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3238,2996,405,457,577,585,457,496,448,419,324,379,400,377,292,213,157,108,68,50,19,2,1,679,5513,578,26,5,0,0.0,0.0,32,13,25,564,14,65,461,36,2,05885,Antioquia,Yalí,05
1118,1.0,0.0,1.0,0.0,0.0,0.0,1.0,17638,18699,2286,2816,3577,3585,3201,2890,2710,2697,2057,2124,1959,1842,1480,1089,760,566,385,234,61,12,6,5244,30374,4338,404,80,4,0.0,1.0,183,190,41,4142,196,705,2564,578,295,05887,Antioquia,Yarumal,05
1119,2.0,0.0,1.0,0.0,0.0,1.0,1.0,9700,9310,1129,1396,1857,1785,1349,1277,1144,1267,1077,1164,1218,1156,948,739,560,400,294,163,62,17,8,1277,17557,970,58,49,5,0.0,4.0,91,62,38,956,14,128,702,94,32,05890,Antioquia,Yolombó,05
1120,3.0,1.0,0.0,0.0,0.0,2.0,1.0,9046,8551,1660,1746,1755,1655,1350,1484,1374,1341,1056,933,831,703,533,377,294,237,156,66,33,7,6,2018,15382,1410,247,68,0,0.0,1.0,173,57,62,1375,34,121,873,296,85,05893,Antioquia,Yondó,05


### Cobertura de aseguramiento

Para complementar el análisis se utilizará la cobertura de aseguramiento reportada por el <a href="{https://www.minsalud.gov.co/proteccionsocial/Regimensubsidiado/Paginas/coberturas-del-regimen-subsidiado.aspx}">Ministerio de Salud</a>

In [12]:
#Importar base de datos régimen subsidiado
columnas = ["CODIGOS DANE", "2018"]

subsidiado = pd.read_excel('Cobertura.xlsx', sheet_name='AfilRS_mpio', usecols = columnas, converters = {"CODIGOS DANE" : str})
subsidiado.rename(columns = { "2018":"subsidiado"}, inplace= True)

#Importar base de datos régimen contributivo
contributivo = pd.read_excel('Cobertura.xlsx', sheet_name='AfilRC_mpio', usecols = columnas, converters = \
                             {"CODIGOS DANE" : str})
contributivo.rename(columns = { "2018":"contributivo"}, inplace= True)

#Unión de bases de datos de los regímenes
cobertura = pd.merge(subsidiado, contributivo, how='inner', on = ["CODIGOS DANE"])
cobertura[["subsidiado", "contributivo"]] = cobertura[["subsidiado", "contributivo"]].fillna(value=0)

#Total de cobertura
cobertura["cobertura"] = (cobertura["subsidiado"] + cobertura["contributivo"]) 

#Eliminar 91000 porque no es un municipio, representaría el depto
cobertura.drop([0], inplace = True)

#Exportar base de datos
cobertura.to_excel("coberturaF.xlsx",index = False)


#Aseguramiento e IPS
cobertura = pd.merge(data_salud, cobertura, how='right', left_on = ["codmunicipio"], \
                    right_on = ["CODIGOS DANE"] )
#Limpieza de nulos en IPS (si es nulo es porque el sistema no tiene información por lo cual se podría pensar que no tiene
#IPS)
cobertura[['cantidad_ips', 'nivel_1','nivel_2', 'nivel_3', 'ips_mixta', 'ips_privada', 'ips_publica']] = \
cobertura[['cantidad_ips', 'nivel_1','nivel_2', 'nivel_3', 'ips_mixta', 'ips_privada', 'ips_publica']].fillna(value=0)

#labels 
#Unión con divipola para cambiar labels de los municipios
cobertura = pd.merge(cobertura, divipola, how='left', left_on = ["CODIGOS DANE"], right_on = ["DIVIPOLA"] )
cobertura

,coddepartamento,codmunicipio,cantidad_ips,nivel_1,nivel_2,nivel_3,ips_mixta,ips_privada,ips_publica,CODIGOS DANE,subsidiado,contributivo,cobertura,DEPARTAMENTO,MUNICIPIO,DEPTO,MUNICIPIO.1,DIVIPOLA
0,91,91001,13.0,2.0,1.0,0.0,0.0,11.0,2.0,91001,33892.0,14918.0,48810.0,Amazonas,Leticia,91,001,91001
1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91263,0.0,14.0,14.0,Amazonas,El Encanto,91,263,91263
2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91405,0.0,23.0,23.0,Amazonas,La Chorrera,91,405,91405
3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91407,0.0,19.0,19.0,Amazonas,La Pedrera,91,407,91407
4,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91430,0.0,0.0,0.0,Amazonas,La Victoria,91,430,91430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97889,0.0,0.0,0.0,Vaupes,Yavaraté,97,889,97889
1118,99,99001,5.0,0.0,1.0,0.0,0.0,4.0,1.0,99001,15357.0,4775.0,20132.0,Vichada,Puerto Carreño,99,001,99001
1119,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99524,7569.0,1244.0,8813.0,Vichada,La Primavera,99,524,99524
1120,99,99624,1.0,0.0,0.0,0.0,0.0,1.0,0.0,99624,2480.0,559.0,3039.0,Vichada,Santa Rosalía,99,624,99624


### Análisis


In [13]:
# Municipios con al menos una IPS
print("Municipios con al menos 1 IPS", sum(df_final.cantidad_ips > 0))
print("Municipios sin IPS", sum(df_final.cantidad_ips == 0))

# Municipios con al menos una persona asegurada
print("Municipios con cobertura de aseguramiento", sum(cobertura.cobertura > 0))
print("Municipios sin personas aseguradas", sum(cobertura.cobertura == 0))

#Municipios sin aseguramiento ni IPS
cobertura["aux"] = cobertura["cobertura"] + cobertura["cantidad_ips"]
print("Municipios sin personas aseguradas ni IPS", sum(cobertura.aux == 0))
aux2 = cobertura[cobertura['aux'] == 0]

#Municipios con IPS privadas,públicas y mixtas
print("Municipios IPS privadas", sum(df_final.ips_privada > 0))
print("Municipios IPS públicas", sum(df_final.ips_publica > 0))
print("Municipios IPS mixtas", sum(df_final.ips_mixta > 0))

print("IPS privadas", sum(df_final.ips_privada))
print("IPS públicas", sum(df_final.ips_publica))
print("IPS mixtas", sum(df_final.ips_mixta))



Municipios con al menos 1 IPS 926
Municipios sin IPS 196
Municipios con cobertura de aseguramiento 1110
Municipios sin personas aseguradas 12
Municipios sin personas aseguradas ni IPS 12
Municipios IPS privadas 443
Municipios IPS públicas 854
Municipios IPS mixtas 17
IPS privadas 9904.0
IPS públicas 1003.0
IPS mixtas 18.0


In [14]:
#Municipios con IPS de nivel 1, 2 y 3 
print("Municipios IPS nivel 1", sum(df_final.nivel_1 > 0))
print("Municipios IPS nivel 2", sum(df_final.nivel_2 > 0))
print("Municipios IPS nivel 3", sum(df_final.nivel_3 > 0))

print("IPS nivel 1", sum(df_final.nivel_1))
print("IPS nivel 2", sum(df_final.nivel_2))
print("IPS nivel 3", sum(df_final.nivel_3))

Municipios IPS nivel 1 798
Municipios IPS nivel 2 106
Municipios IPS nivel 3 19
IPS nivel 1 859.0
IPS nivel 2 115.0
IPS nivel 3 28.0
